In [1]:
import sys
sys.path.append('../python/')

In [2]:
import pandas as pd
import cdr
import json
import imp

In [3]:
imp.reload(cdr)

<module 'cdr' from '../python/cdr.py'>

In [4]:
df = pd.read_csv('../../data/CDR/sms-call-internet-mi-2013-11-01.txt', delimiter='\t', header=None) 
df2 = pd.read_csv('../../data/CDR/sms-call-internet-mi-2013-11-02.txt', delimiter='\t', header=None) 
file = '../../data/GeoJSON/milano-grid.geojson'
with open(file) as f:
    grid = pd.read_json(f, typ='Series')

In [5]:
m_weekday, d_weekday, e_weekday, m_weekend, d_weekend, e_weekend = cdr.join_cdr_grid_by_time(df, grid)
m_weekday2, d_weekday2, e_weekday2, m_weekend2, d_weekend2, e_weekend2 = cdr.join_cdr_grid_by_time(df2, grid)
m_weekday.fillna(0, inplace=True)
m_weekday2.fillna(0, inplace=True)
cols = ['cellId','smsIn','smsOut','callIn','callOut','internet']

In [27]:
asb = pd.concat([m_weekday, m_weekday2])

In [28]:
asb= asb.groupby('properties.cellId').agg({
                        'properties.cellId': 'first', 
                        'geometry.coordinates': 'first',
                        'geometry.type': 'first',
                        'id': 'first',
                        'cellId': 'first',
                        'type': 'first',
                        'smsIn': 'sum',
                        'smsOut': 'sum',
                        'callIn': 'sum',
                        'callOut': 'sum',
                        'internet': 'sum'})

In [29]:
asb.head()

,internet,cellId,id,geometry.type,smsOut,callOut,geometry.coordinates,callIn,type,properties.cellId,smsIn
properties.cellId,,,,,,,,,,,
1,320.643279,1.0,0,Polygon,4.137999,1.510645,"[[[9.0114910478323, 45.35880131440966], [9.014...",1.305744,Feature,1,5.450417
2,321.442663,2.0,1,Polygon,4.145378,1.539763,"[[[9.014491488013135, 45.35880097314403], [9.0...",1.323735,Feature,2,5.477261
3,322.293575,3.0,2,Polygon,4.153233,1.570757,"[[[9.017491928134044, 45.358800553060284], [9....",1.342885,Feature,3,5.505834
4,318.327848,4.0,3,Polygon,4.116626,1.426305,"[[[9.02049236818262, 45.35880005415845], [9.02...",1.253634,Feature,4,5.372665
5,290.004421,5.0,4,Polygon,3.739232,1.326145,"[[[9.023492808146456, 45.35879947643852], [9.0...",1.174183,Feature,5,4.869727


In [21]:
m_weekday.head()

,geometry.coordinates,geometry.type,id,properties.cellId,type,internet,cellId,smsOut,callOut,time,callIn,smsIn
0,"[[[9.0114910478323, 45.35880131440966], [9.014...",Polygon,0,1,Feature,320.643279,1,4.137999,1.510645,1383260400000,1.305744,5.450417
1,"[[[9.014491488013135, 45.35880097314403], [9.0...",Polygon,1,2,Feature,321.442663,2,4.145378,1.539763,1383260400000,1.323735,5.477261
2,"[[[9.017491928134044, 45.358800553060284], [9....",Polygon,2,3,Feature,322.293575,3,4.153233,1.570757,1383260400000,1.342885,5.505834
3,"[[[9.02049236818262, 45.35880005415845], [9.02...",Polygon,3,4,Feature,318.327848,4,4.116626,1.426305,1383260400000,1.253634,5.372665
4,"[[[9.023492808146456, 45.35879947643852], [9.0...",Polygon,4,5,Feature,290.004421,5,3.739232,1.326145,1383260400000,1.174183,4.869727


In [30]:
# reorganizing them  to GeoJSON    
cols = ['cellId','smsIn','smsOut','callIn','callOut','internet']

m_wd = cdr.df_to_geojson(asb, cols)
# d_wd = cdr.df_to_geojson(d_weekday, cols)
# e_wd = cdr.df_to_geojson(e_weekday, cols)
# m_we = cdr.df_to_geojson(m_weekend, cols)
# d_we = cdr.df_to_geojson(d_weekend, cols)
# e_we = cdr.df_to_geojson(e_weekend, cols)

In [32]:
# writing the file
output_filename = '../../data/CDR/generated/CDR_morning_weekday.geojson'
with open(output_filename, 'w') as output_file:
   json.dump(m_wd, output_file, indent=2)

output_filename = '../../data/CDR/generated/CDR_day_weekday.geojson'
with open(output_filename, 'w') as output_file:
   json.dump(d_wd, output_file, indent=2)

output_filename = '../../data/CDR/generated/CDR_evening_weekday.geojson'
with open(output_filename, 'w') as output_file:
   json.dump(e_wd, output_file, indent=2)

output_filename = '../../data/CDR/generated/CDR_morning_weekend.geojson'
with open(output_filename, 'w') as output_file:
   json.dump(m_we, output_file, indent=2)

output_filename = '../../data/CDR/generated/CDR_day_weekend.geojson'
with open(output_filename, 'w') as output_file:
   json.dump(d_we, output_file, indent=2)

output_filename = '../../data/CDR/generated/CDR_evening_weekend.geojson'
with open(output_filename, 'w') as output_file:
   json.dump(e_we, output_file, indent=2)

print ("file generated.")

file generated.
